In [1]:
from pynq import Overlay
from pynq import allocate
import math
import time
import numpy as np

In [2]:
# Load Overlay
overlay = Overlay('./Overlay/cnna.bit')

In [3]:
overlay?

In [4]:
test_ip = overlay.cnna_0

In [5]:
test_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  in_data_1 = Register(in_data=0),
  in_data_2 = Register(in_data=0),
  in_weights_1 = Register(in_weights=0),
  in_weights_2 = Register(in_weights=0),
  out_r_1 = Register(out_r=0),
  out_r_2 = Register(out_r=0)
}

```c
// control
// 0x00 : Control signals
//        bit 0  - ap_start (Read/Write/COH)
//        bit 1  - ap_done (Read/COR)
//        bit 2  - ap_idle (Read)
//        bit 3  - ap_ready (Read)
//        bit 7  - auto_restart (Read/Write)
//        others - reserved
// 0x04 : Global Interrupt Enable Register
//        bit 0  - Global Interrupt Enable (Read/Write)
//        others - reserved
// 0x08 : IP Interrupt Enable Register (Read/Write)
//        bit 0  - enable ap_done interrupt (Read/Write)
//        bit 1  - enable ap_ready interrupt (Read/Write)
//        others - reserved
// 0x0c : IP Interrupt Status Register (Read/TOW)
//        bit 0  - ap_done (COR/TOW)
//        bit 1  - ap_ready (COR/TOW)
//        others - reserved
// 0x10 : Data signal of in_data
//        bit 31~0 - in_data[31:0] (Read/Write)
// 0x14 : Data signal of in_data
//        bit 31~0 - in_data[63:32] (Read/Write)
// 0x18 : reserved
// 0x1c : Data signal of in_weights
//        bit 31~0 - in_weights[31:0] (Read/Write)
// 0x20 : Data signal of in_weights
//        bit 31~0 - in_weights[63:32] (Read/Write)
// 0x24 : reserved
// 0x28 : Data signal of out_r
//        bit 31~0 - out_r[31:0] (Read/Write)
// 0x2c : Data signal of out_r
//        bit 31~0 - out_r[63:32] (Read/Write)
// 0x30 : reserved
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)

```

In [6]:
# Method for FP and FxP conversion.
def FloatToFix(infloat, outW, outI, verbose=False):
    # Convert integer part
    inint = math.floor(abs(infloat))
    binint = bin(inint).replace("0b","")
    if(math.floor(abs(infloat)) >= 2**(outI-1)):
        raise Exception("Number too large for integer width")
    while len(binint) < (outI): 
        binint = '0' + binint
        
    # Convert decimal part
    indec = abs(infloat) - math.floor(abs(infloat))
    bindec = ""
    i=0
    while (len(bindec) < (outW - outI)):
        i -= 1
        if(indec >= (2**(i))):
            indec -= 2**(i)
            bindec += '1'
        else:
            bindec += '0'   
    outbin = binint + bindec # Unsigned
    print("Float -> Fix:", infloat, " |", binint, bindec) if verbose else None
    print("Before TC:", outbin) if verbose else None
    
    # Twos Complement
    # Convert if negative
    if(infloat < 0):
        newbin = ''
        # Flip everything
        for ec in outbin:
            if(ec == '0'):
                newbin += '1'
            else:
                newbin += '0'
        outbin = newbin
        # Flip tail until first 0 (add 1)
        newbin = ''
        addFlip = True
        for i in range(outW):
            if(outbin[-(i+1)] == '0'):
                if addFlip:
                    newbin = '1' + newbin
                else:
                    newbin = '0' + newbin
                # Stop Flipping
                addFlip = False
            else:
                if addFlip:
                    newbin = '0' + newbin
                else:
                    newbin = '1' + newbin
        outbin = newbin
    # End TC Conversion
    print("After TC: ", outbin) if verbose else None
    
    outdec=0
    for i in range(outW):
        if(outbin[-(i+1)] == '1'):
            outdec += (2**i)
    
    #print(outdec, outbin)
    return outdec

In [7]:
FloatToFix(0.5, 8, 1, verbose=True)

Float -> Fix: 0.5  | 0 1000000
Before TC: 01000000
After TC:  01000000


64

In [8]:
def FixToFloat(inFix, inW, inI, verbose=False):
    inbin = bin(inFix).replace("0b", "")
    while len(inbin) < inW:
        inbin = '0'+ inbin
    print("Fix -> Float:", inFix, " |", inbin[:inI], inbin[inI:]) if verbose else None
    
    neg = False
    print("Before TC:", inbin)
    if(inbin[0] == '1'):
        neg = True
        newbin = ''
        # Flip everything
        for ec in inbin:
            if(ec == '0'):
                newbin += '1'
            else:
                newbin += '0'
        inbin = newbin
        # Flip tail until first 0 (add 1)
        newbin = ''
        addFlip = True
        for i in range(inW):
            if(inbin[-(i+1)] == '0'):
                if addFlip:
                    newbin = '1' + newbin
                else:
                    newbin = '0' + newbin
                # Stop Flipping
                addFlip = False
            else:
                if addFlip:
                    newbin = '0' + newbin
                else:
                    newbin = '1' + newbin
        inbin = newbin
    print("After TC:", inbin)
    
    binint = inbin[:inI]
    binint = binint[::-1]
    outI = 0
    for i in range(len(binint)):
        if(binint[i] == '1'):
            outI += 2**i
    
    bindec = inbin[inI:]
    outD = 0
    for i in range(len(bindec)):
        if(bindec[i] == '1'):
            outD += 2**(-i-1)
    out = outI + outD
    if neg:
        out = out * -1
    #print(outI+outD, inbin)
    return out

In [9]:
FixToFloat(64, 8, 1, verbose=True)

Fix -> Float: 64  | 0 1000000
Before TC: 01000000
After TC: 01000000


0.5

In [10]:
in1 = allocate(shape=(3,3,3), cacheable=1, dtype='uint16')
in2 = allocate(shape=(3,3,3), cacheable=1, dtype='uint8')
print(in1.shape)
#print(in1.physical_address)
print(in2.shape)
#print(in2.physical_address)

(3, 3, 3)
(3, 3, 3)


In [11]:
# ap_fixed<16,6> 0000 00.00 0000 0000 
print("0.1 =>", FloatToFix(0.1, 16, 6))
print("0.2 =>", FloatToFix(0.2, 16, 6))
print("0.3 =>", FloatToFix(0.3, 16, 6))
print("0.5 =>", FloatToFix(0.5, 16, 6))
print("1 =>", FloatToFix(1, 16, 6))
print("1.5 =>", FloatToFix(2.5, 16, 6))
print("2 =>", FloatToFix(2, 16, 6))
print("-1 =>", FloatToFix(-1, 16, 6))
print("-1.5 =>", FloatToFix(-1.5, 16, 6))
print("-2 =>", FloatToFix(-2, 16, 6))

0.1 => 102
0.2 => 204
0.3 => 307
0.5 => 512
1 => 1024
1.5 => 2560
2 => 2048
-1 => 64512
-1.5 => 64000
-2 => 63488


In [12]:

tarr1 = np.array([
    [[1024,1024,1024],[2048,2048,2048],[2560,2560,2560]],
    [[64512,64000,63488],[64512,64000,63488],[64512,64000,63488]],
    [[1024,2560,2048],[64512,64000,63488],[1024,2560,2048]]], dtype=np.uint16)

tarr2 = np.full((3,3,3), FloatToFix(0.5, 8, 1), dtype=np.uint8)


print(tarr1.shape)
print(tarr1)
print("= = = = =")
print(tarr2.shape)
print(tarr2)

(3, 3, 3)
[[[ 1024  1024  1024]
  [ 2048  2048  2048]
  [ 2560  2560  2560]]

 [[64512 64000 63488]
  [64512 64000 63488]
  [64512 64000 63488]]

 [[ 1024  2560  2048]
  [64512 64000 63488]
  [ 1024  2560  2048]]]
= = = = =
(3, 3, 3)
[[[64 64 64]
  [64 64 64]
  [64 64 64]]

 [[64 64 64]
  [64 64 64]
  [64 64 64]]

 [[64 64 64]
  [64 64 64]
  [64 64 64]]]


In [13]:
np.copyto(in1, tarr1)
np.copyto(in2, tarr2)

In [14]:
#for i in range(3):
#    for j in range(3):
#        for k in range(3):
#            in1[i][j][k] = tarr1[i][j][k]
#            in2[i][j][k] = tarr2[i][j][k]

In [15]:
print(in1)
print('\n= = = = = =\n')
print(in2)

[[[ 1024  1024  1024]
  [ 2048  2048  2048]
  [ 2560  2560  2560]]

 [[64512 64000 63488]
  [64512 64000 63488]
  [64512 64000 63488]]

 [[ 1024  2560  2048]
  [64512 64000 63488]
  [ 1024  2560  2048]]]

= = = = = =

[[[64 64 64]
  [64 64 64]
  [64 64 64]]

 [[64 64 64]
  [64 64 64]
  [64 64 64]]

 [[64 64 64]
  [64 64 64]
  [64 64 64]]]


In [16]:
# Allocate memory to store output
out = allocate(shape=(3,3), dtype='uint16')
print(out)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [17]:
in1s = 0x10
in2s = 0x1c
outs = 0x28

t1 = time.time()

i = 0

'''
while i < 9:
    if(i%4 == 0):
        print(">at offset",hex(i))
        if(i+4 < in1.shape[0]):
            tdat = in1[i:i+4].tobytes()
        else:
            tdat = in1[i:].tobytes()
            
        print(tdat)
        test_ip.write(in1s+i, tdat)
        
        if(i+4 < in1.shape[0]):
            tdat = in2[i:i+4].tobytes()
        else:
            tdat = in2[i:].tobytes()
            
        print(tdat)
        test_ip.write(in2s+i, tdat)
    i += 1
'''


#test_ip.register_map.Memory_in_data = in1
#test_ip.register_map.Memory_in_weights = in2

test_ip.write(in1s, in1.physical_address)
test_ip.write(in2s, in2.physical_address)
test_ip.write(outs, out.physical_address)
test_ip.write(0x00, 1)

t2 = time.time()

print(t2-t1)
print("out:", out)

0.0012135505676269531
out: [[ 1536  3072  3840]
 [63232 63232 63232]
 [ 2816 63232  2816]]


In [18]:
FixToFloat(3072, 16, 6)

Before TC: 0000110000000000
After TC: 0000110000000000


3